In [1]:
import json_lines
import numpy as np
import torch

from datasets import Dataset
from collections import defaultdict
from sklearn.metrics import accuracy_score

from peft import LoraConfig, get_peft_model, TaskType
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq

In [2]:
def get_riddlesense_prompt(question, options):
    prompt = \
"""
Question: {}

What is the correct answer to the question from the following choices?
Options: 
(A): {}
(B): {}
(C): {}
(D): {}
(E): {}""".format(question, options[0], options[1], options[2], options[3], options[4])
    return prompt

In [3]:
def load_data(file_path):
    raw_data = []
    with open(file_path, 'rb') as f: 
        for item in json_lines.reader(f):
            raw_data.append(item)
    
    data = defaultdict(list)
    for item in raw_data:
        data['question'].append(item['question']['stem'])
        data['options'].append([_['text'] for _ in item['question']['choices']])
        data['answer'].append(item['answerKey'])
    return data

train_data = load_data("data/rs_train.jsonl")
valid_data = load_data("data/rs_dev.jsonl")

In [4]:
def preprocess_function(sample):
    text = get_riddlesense_prompt(sample['question'], sample['options'])

    model_inputs = tokenizer(
        text,
        truncation=True,
        max_length=512,
        padding='max_length',
        return_attention_mask=True,
    )

    labels = tokenizer(sample['answer'], max_length=2, padding="max_length", truncation=True)
    labels = labels["input_ids"]
    labels = [l if l != tokenizer.pad_token_id else -100 for l in labels]
    model_inputs["labels"] = labels

    return model_inputs

In [5]:
model_name = 'google/flan-t5-large'
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define LoRA Config
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
lora_config = LoraConfig(
    r=4,
    lora_alpha=32,
    target_modules=["q", "v"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)

# add LoRA adaptor
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 1,179,648 || all params: 784,329,728 || trainable%: 0.15040205131686657


In [6]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

1179648

In [7]:
train_dataset = Dataset.from_dict(train_data)
train_tokenized = train_dataset.map(preprocess_function, batched=False, remove_columns=['question', 'options', 'answer'])
print(train_tokenized)

valid_dataset = Dataset.from_dict(valid_data)
valid_tokenized = valid_dataset.map(preprocess_function, batched=False, remove_columns=['question', 'options', 'answer'])
print(valid_tokenized)

Map:   0%|          | 0/3510 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 3510
})


Map:   0%|          | 0/1021 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 1021
})


In [8]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100

# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=2
)

In [9]:
def compute_metrics(p):
    predictions, labels = p
    # https://discuss.huggingface.co/t/what-does-evalprediction-predictions-contain-exactly/1691/4
    logits = predictions[0]
    predictions = np.argmax(logits, axis=2)
    
    true_predictions = [
        [p for (p, l) in zip(prediction, label) if l != -100 and l != 1]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [l for (p, l) in zip(prediction, label) if l != -100 and l != 1]
        for prediction, label in zip(predictions, labels)
    ]

    results = accuracy_score(y_true=true_labels, y_pred=true_predictions)
    return {
        "accuracy": results,
    }

In [10]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir = f"/usr1/data/devanshj/brainteaser/checkpoints/{model_name[7:]}_rs_lora"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    learning_rate=3e-4, # higher learning rate
	per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    load_best_model_at_end=True,
    push_to_hub=True
)


# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    eval_dataset=valid_tokenized,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
)

In [11]:
trainer.train()
trainer.push_to_hub()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/home/devanshj/miniconda3/envs/anlp/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

: 

In [12]:
peft_model_id = f"{model_name[7:]}_rs_lora"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)

('flan-t5-small_rs_lora/tokenizer_config.json',
 'flan-t5-small_rs_lora/special_tokens_map.json',
 'flan-t5-small_rs_lora/tokenizer.json')

## Inference

In [13]:
def get_brainteaser_prompt(question, options):
    prompt = \
"""
Question: {}

What is the correct answer to the question from the following choices?
Options: 
(A): {}
(B): {}
(C): {}
(D): {}""".format(question, options[0], options[1], options[2], options[3])
    return prompt

In [14]:
question = "Mr. and Mrs. Mustard have six daughters and each daughter has one brother. But there are only 9 people in the family, how is that possible?"
options = ["Some daughters get married and have their own family.", "Each daughter shares the same brother.", "Some brothers were not loved by family and moved away.", "None of above."]
bt_prompt = get_brainteaser_prompt(question, options)

In [15]:
inputs = tokenizer(bt_prompt, return_tensors="pt")

In [16]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['C']


In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)
model.eval()

print("LoRA loaded")